In [1]:
import pandas as pd

In [14]:
df = pd.read_csv("D:\OneDrive - NITT\Custom_Download\exported.csv")

In [15]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [16]:
df.groupby('year').count()

,count,name_g
year,,
1937,8948,8948
1938,9031,9031
1939,8918,8918
1940,8962,8962
1941,9086,9086
...,...,...
2018,32221,32221
2019,32109,32109
2020,31517,31517


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode the categorical column 'name_g'
label_encoder = LabelEncoder()
df['name_g_encoded'] = label_encoder.fit_transform(df['name_g'])
# Split the data into features (X) and target variable (y)
X = df[['year', 'name_g_encoded']]
y = df['count']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train

,year,name_g_encoded
667916,1964,79366
1739696,2019,80316
1644430,2022,354
707898,1965,35677
1003653,1990,74777
...,...,...
259178,2016,38941
1414414,1994,61869
131932,2001,97530
671155,1964,99274


In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input

# Sample DataFrame
data = {'year': [2000, 2000, 2000, 2000, 2000, 2019, 2019, 2019, 2019, 2019],
        'count': [25957, 23085, 19968, 17998, 17712, 5, 5, 5, 5, 5],
        'name_g': ['Emily_F', 'Hannah_F', 'Madison_F', 'Ashley_F', 'Sarah_F',
                   'Zyheem_M', 'Zykel_M', 'Zyking_M', 'Zyn_M', 'Zyran_M']}
df = pd.DataFrame(data)

# Convert Year column to datetime and set it as the index
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index('year', inplace=True)

# Normalize the count values
scaler_count = MinMaxScaler(feature_range=(0, 1))
scaled_count = scaler_count.fit_transform(df[['count']])

# Encode the name_g column
df['name_g_encoded'] = df['name_g'].astype('category').cat.codes

# Normalize the encoded name values
scaler_name = MinMaxScaler(feature_range=(0, 1))
scaled_name = scaler_name.fit_transform(df[['name_g_encoded']])

# Normalize the year values
scaler_year = MinMaxScaler(feature_range=(0, 1))
scaled_year = scaler_year.fit_transform(df[['year']])

# Combine the scaled count, name, and year features
scaled_data = np.concatenate([scaled_count, scaled_name, scaled_year], axis=1)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

# Prepare data for LSTM model
def create_dataset(data, time_steps):
    X, y_count, y_name, y_year = [], [], [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y_count.append(data[i + time_steps, 0])
        y_name.append(data[i + time_steps, 1])
        y_year.append(data[i + time_steps, 2])
    return np.array(X), np.array(y_count), np.array(y_name), np.array(y_year)

time_steps = 12  # Define the number of time steps
X_train, y_train_count, y_train_name, y_train_year = create_dataset(train_data, time_steps)
X_test, y_test_count, y_test_name, y_test_year = create_dataset(test_data, time_steps)

# Define the LSTM model architecture
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
lstm_layer = LSTM(units=50)(input_layer)
dense_count = Dense(units=1, name='count')(lstm_layer)
dense_name = Dense(units=1, name='name')(lstm_layer)
dense_year = Dense(units=1, name='year')(lstm_layer)

# Compile the model
model = Model(inputs=input_layer, outputs=[dense_count, dense_name, dense_year])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, [y_train_count, y_train_name, y_train_year], epochs=100, batch_size=32, validation_data=(X_test, [y_test_count, y_test_name, y_test_year]))

# Evaluate the model
loss = model.evaluate(X_test, [y_test_count, y_test_name, y_test_year])
print("Test Loss:", loss)

# Make predictions
predictions = model.predict(X_test)


ModuleNotFoundError: No module named 'tensorflow'

In [11]:
!pip install tensorflow